### Just change the code to predict oof feature individually for each type 

In [17]:
import numpy as np
import pandas as pd
import feather
from sklearn.model_selection import GroupKFold
import lightgbm as lgb
import gc

import sys
sys.path.append("./../")
from utils import reduce_mem_usage

In [2]:
#実際予測時にはその化合物のデータは使えないので、厳密にはGroupKfoldで化合物ごとに分けるべき

In [3]:
features = [
        "Atom",
        "AtomPosition",
        "AtomDistance",
        "CouplingType",
        "AtomEnvironment",
        "AtomNeighbors",
        "BruteForce",
        "DistanceFromClosest",
        "ElectroNegFromClosest",
        "ACSF",
        "MullikenChargePred",
        "OpenBabelBasic"]

In [4]:
X_train = pd.concat([feather.read_dataframe("../features/" + feature + "_train.feather") for feature in features], axis=1)
X_test = pd.concat([feather.read_dataframe("../features/" + feature + "_test.feather") for feature in features], axis=1)

In [5]:
X_train.shape, X_test.shape

((4658147, 452), (2505542, 452))

In [6]:
X_train.head()

,atom_0,atom_1,x_0,x_1,y_0,y_1,z_0,z_1,atom_distance,dist_inv_0_sum,...,tertiary_angle_6,tertiary_atom_7,tertiary_distance_7,tertiary_angle_7,tertiary_atom_8,tertiary_distance_8,tertiary_angle_8,tertiary_atom_9,tertiary_distance_9,tertiary_angle_9
0,H,C,0.002150,-0.012698,-0.006031,1.085804,0.001976,0.008001,1.091953,1.297179,...,NaN,None,NaN,NaN,None,NaN,NaN,None,NaN,NaN
1,H,H,0.002150,1.011731,-0.006031,1.463751,0.001976,0.000277,1.783120,1.297179,...,NaN,None,NaN,NaN,None,NaN,NaN,None,NaN,NaN
2,H,H,0.002150,-0.540815,-0.006031,1.447527,0.001976,-0.876644,1.783147,1.297179,...,NaN,None,NaN,NaN,None,NaN,NaN,None,NaN,NaN
3,H,H,0.002150,-0.523814,-0.006031,1.437933,0.001976,0.906397,1.783157,1.297179,...,NaN,None,NaN,NaN,None,NaN,NaN,None,NaN,NaN
4,H,C,1.011731,-0.012698,1.463751,1.085804,0.000277,0.008001,1.091952,1.120798,...,NaN,None,NaN,NaN,None,NaN,NaN,None,NaN,NaN


In [7]:
contri = feather.read_dataframe("../data/input/scalar_coupling_contributions.feather")

In [8]:
contri.head()

,molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso,dso
0,dsgdb9nsd_000001,1,0,1JHC,83.0224,0.254579,1.25862,0.272010
1,dsgdb9nsd_000001,1,2,2JHH,-11.0347,0.352978,2.85839,-3.433600
2,dsgdb9nsd_000001,1,3,2JHH,-11.0325,0.352944,2.85852,-3.433870
3,dsgdb9nsd_000001,1,4,2JHH,-11.0319,0.352934,2.85855,-3.433930
4,dsgdb9nsd_000001,2,0,1JHC,83.0222,0.254585,1.25861,0.272013


In [10]:
# keep nan as is
#coupling_types = X_train['type']
categorical_cols = list(X_train.columns[X_train.dtypes == object])


for col in categorical_cols:    
    print(f'Starting {col}')
    uniques = list(X_train[col].unique())
    if None in uniques:
        uniques.remove(None)
    mapping = dict(zip(uniques, range(1, len(uniques)+1)))
    X_train[col] = X_train[col].map(mapping)
    X_test[col] = X_test[col].map(mapping)

Starting atom_0
Starting atom_1
Starting type
Starting a1_is_aromatic
Starting a1_in_ring
Starting a1_in_ring3
Starting a1_in_ring4
Starting a1_in_ring5
Starting a1_in_ring6
Starting a1_in_ring7
Starting a1_in_ring8
Starting bond_atom
Starting tertiary_atom_0
Starting tertiary_atom_1
Starting tertiary_atom_2
Starting tertiary_atom_3
Starting tertiary_atom_4
Starting tertiary_atom_5
Starting tertiary_atom_6
Starting tertiary_atom_7
Starting tertiary_atom_8
Starting tertiary_atom_9


In [16]:
X_train = reduce_mem_usage(X_train)
X_test = reduce_mem_usage(X_test)

Memory usage of dataframe is 15104.01 MB
Memory usage after optimization is: 7934.05 MB
Decreased by 47.5%
Memory usage of dataframe is 8114.64 MB
Memory usage after optimization is: 4250.87 MB
Decreased by 47.6%


NameError: name 'gc' is not defined

In [18]:
gc.collect()

1330

In [35]:
train_types = X_train['type'].values
test_types = X_test['type'].values

In [43]:
oof_train = pd.DataFrame()
oof_test = pd.DataFrame()


for col in ['fc', 'sd', 'pso', 'dso']:
    
    target = contri[col]
    
    oof_type = np.zeros(len(X_train))
    pred_type = np.zeros(len(X_test))
    
    for cur_type in np.unique(train_types):
        cur_type_idx_train = (train_types == cur_type)
        cur_type_idx_test = (test_types == cur_type)

        cur_type_train = X_train.iloc[cur_type_idx_train]
        cur_type_test = X_test.iloc[cur_type_idx_test]
        cur_type_target = target.iloc[cur_type_idx_train]
        cur_type_mols = contri['molecule_name'].iloc[cur_type_idx_train]
        
    
        kf = GroupKFold(n_splits=2)
    
        oof = np.zeros(len(cur_type_train))
        pred = np.zeros(len(cur_type_test))
        
        
        SEED = 42
        NUM_ROUNDS = 10000
        
        params = {
                    "num_leaves": 100,
                    "min_data_in_leaf": 100,
                    "objective": "regression",
                    "max_depth": 10,
                    "learning_rate": 0.2,
                    "boosting_type": "gbdt",
                    "subsample_freq": 1,
                    "subsample": 0.9,
                    "metric": "mae",
                    "reg_alpha": 0.1,
                    "reg_lambda": 0.3, 
                    "colsample_bytree": 0.9
                    }

        for train_idx, val_idx in kf.split(cur_type_train, groups=cur_type_mols):
            train_data = lgb.Dataset(cur_type_train.iloc[train_idx], label=cur_type_target.iloc[train_idx], categorical_feature=categorical_cols)
            val_data = lgb.Dataset(cur_type_train.iloc[val_idx], label=cur_type_target.iloc[val_idx], categorical_feature=categorical_cols)
            clf = lgb.train(params, train_data, NUM_ROUNDS, valid_sets=[train_data, val_data],
                            verbose_eval=1000, early_stopping_rounds=100)
            oof[val_idx] = clf.predict(cur_type_train.iloc[val_idx], num_iteration=clf.best_iteration)
            pred += clf.predict(cur_type_test, num_iteration=clf.best_iteration) / kf.n_splits

        oof_type[cur_type_idx_train] = oof
        pred_type[cur_type_idx_test] = pred
        
        print(f'{cur_type} for {col} done!')
        
    oof_train[col] = oof_type
    oof_test[col] = pred_type

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.406676	valid_1's l1: 0.759535
[2000]	training's l1: 0.23861	valid_1's l1: 0.722005
[3000]	training's l1: 0.1532	valid_1's l1: 0.709364
[4000]	training's l1: 0.10261	valid_1's l1: 0.704061
[5000]	training's l1: 0.0706691	valid_1's l1: 0.701116
[6000]	training's l1: 0.0495137	valid_1's l1: 0.699595
[7000]	training's l1: 0.0354387	valid_1's l1: 0.698623
[8000]	training's l1: 0.0256399	valid_1's l1: 0.698049
[9000]	training's l1: 0.0187593	valid_1's l1: 0.697726
[10000]	training's l1: 0.0139179	valid_1's l1: 0.697502
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.0139179	valid_1's l1: 0.697502
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.408137	valid_1's l1: 0.761197
[2000]	training's l1: 0.239146	valid_1's l1: 0.722948
[3000]	training's l1: 0.153376	valid_1's l1: 0.71021
[4000]	training's l1: 0.102717	valid_1's l1: 0.704546
[5000]	train

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.0647107	valid_1's l1: 0.183165
[2000]	training's l1: 0.0295535	valid_1's l1: 0.178646
[3000]	training's l1: 0.0151506	valid_1's l1: 0.177451
[4000]	training's l1: 0.00834341	valid_1's l1: 0.177074
[5000]	training's l1: 0.00492689	valid_1's l1: 0.17692
[6000]	training's l1: 0.00312233	valid_1's l1: 0.176846
[7000]	training's l1: 0.00212347	valid_1's l1: 0.176808
[8000]	training's l1: 0.00153166	valid_1's l1: 0.176788
[9000]	training's l1: 0.00116391	valid_1's l1: 0.176776
[10000]	training's l1: 0.000923763	valid_1's l1: 0.176769
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.000923763	valid_1's l1: 0.176769
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.0650167	valid_1's l1: 0.182459
[2000]	training's l1: 0.02961	valid_1's l1: 0.177897
[3000]	training's l1: 0.015168	valid_1's l1: 0.176736
[4000]	training's l1: 0.00837468	valid_1's l1: 0

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


...

Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.0102217	valid_1's l1: 0.0201531
[2000]	training's l1: 0.00595021	valid_1's l1: 0.0193563
[3000]	training's l1: 0.00388608	valid_1's l1: 0.0191275
[4000]	training's l1: 0.00271574	valid_1's l1: 0.0190256
[5000]	training's l1: 0.0020024	valid_1's l1: 0.0189796
[6000]	training's l1: 0.00153899	valid_1's l1: 0.0189498
[7000]	training's l1: 0.00122491	valid_1's l1: 0.0189328
[8000]	training's l1: 0.00100493	valid_1's l1: 0.0189226
[9000]	training's l1: 0.000847369	valid_1's l1: 0.0189166
[10000]	training's l1: 0.000731836	valid_1's l1: 0.0189121
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.000731836	valid_1's l1: 0.0189121
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.0103203	valid_1's l1: 0.0205117
[2000]	training's l1: 0.00603846	valid_1's l1: 0.0197324
[3000]	training's l1: 0.00395364	valid_1's l1: 0.019498
[4000]	training's l1: 0.002

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.00665684	valid_1's l1: 0.0091347
[2000]	training's l1: 0.00480556	valid_1's l1: 0.00846906
[3000]	training's l1: 0.00375936	valid_1's l1: 0.0082092
[4000]	training's l1: 0.00304901	valid_1's l1: 0.00807168
[5000]	training's l1: 0.00253739	valid_1's l1: 0.00799111
[6000]	training's l1: 0.0021511	valid_1's l1: 0.00793983
[7000]	training's l1: 0.00184797	valid_1's l1: 0.00790162
[8000]	training's l1: 0.00160625	valid_1's l1: 0.00787361
[9000]	training's l1: 0.00141117	valid_1's l1: 0.00785395
[10000]	training's l1: 0.00125157	valid_1's l1: 0.0078383
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.00125157	valid_1's l1: 0.0078383
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.00669284	valid_1's l1: 0.00914933
[2000]	training's l1: 0.00483312	valid_1's l1: 0.00848649
[3000]	training's l1: 0.00377055	valid_1's l1: 0.00821832
[4000]	training's

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.000803039	valid_1's l1: 0.00273867
[2000]	training's l1: 0.000596695	valid_1's l1: 0.00270824
[3000]	training's l1: 0.000531308	valid_1's l1: 0.00269894
Early stopping, best iteration is:
[3396]	training's l1: 0.000515138	valid_1's l1: 0.00269684
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.00080638	valid_1's l1: 0.00274264
[2000]	training's l1: 0.00059772	valid_1's l1: 0.00271268
[3000]	training's l1: 0.000530615	valid_1's l1: 0.0027041
[4000]	training's l1: 0.000497357	valid_1's l1: 0.00269968
Early stopping, best iteration is:
[3970]	training's l1: 0.000497563	valid_1's l1: 0.00269966
8 for dso done!


In [44]:
oof_train.shape, oof_test.shape

((4658147, 4), (2505542, 4))

In [45]:
oof_train.isnull().sum()

fc     0
sd     0
pso    0
dso    0
dtype: int64

In [46]:
oof_train.to_feather('../features/ScalarCouplingContributionsOof3_train.feather')
oof_test.to_feather('../features/ScalarCouplingContributionsOof3_test.feather')